<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
#| include: false

In [ ]:
#| include: false
from nbdev.showdoc import *

## 0. BaseSubmitter

[`BaseSubmitter`](https://crowdcent.github.io/numerblox/submission.html#basesubmitter) handles all submission logic common to Numerai Classic and Numerai Signals. Under the hood directory logic is handled by [`BaseIO`](https://crowdcent.github.io/numerblox/download.html#baseio).
Each submittor should inherit from [`BaseSubmitter`](https://crowdcent.github.io/numerblox/submission.html#basesubmitter) and implement the `.save_csv` method.

In [1]:
#| echo: false
#| output: asis
show_doc(BaseSubmitter)

---

[source](https://github.com/crowdcent/numerblox/blob/master/numerblox/submission.py#L26){target="_blank" style="float:right; font-size:smaller"}

### BaseSubmitter

>      BaseSubmitter (directory_path:str,
>                     api:Union[numerapi.numerapi.NumerAPI,numerapi.signalsapi.S
>                     ignalsAPI])

Basic functionality for submitting to Numerai. 

Uses numerapi under the hood.
More info: https://numerapi.readthedocs.io/ 

:param directory_path: Directory to store and read submissions from. 

:param api: NumerAPI or SignalsAPI

## 1. NumeraiClassicSubmitter

For Numerai Classic submissions. Uses [NumerAPI](https://numerapi.readthedocs.io/en/latest/_modules/numerapi/numerapi.html) under the hood.

Note that using submitters requires a [`Key`](https://crowdcent.github.io/numerblox/key.html#key) object.

In [2]:
#| echo: false
#| output: asis
show_doc(NumeraiClassicSubmitter)

---

[source](https://github.com/crowdcent/numerblox/blob/master/numerblox/submission.py#L174){target="_blank" style="float:right; font-size:smaller"}

### NumeraiClassicSubmitter

>      NumeraiClassicSubmitter (directory_path:str, key:numerblox.key.Key,
>                               *args, **kwargs)

Submit for Numerai Classic.

:param directory_path: Base directory to save and read prediction files from. 

:param key: Key object containing valid credentials for Numerai Classic. 

*args, **kwargs will be passed to NumerAPI initialization.

### Example usage 1: NumeraiClassicSubmitter

In [ ]:
# example 1
# Initialization (Random credentials)
test_dir = "test_sub"
classic_key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
num_sub = NumeraiClassicSubmitter(directory_path=test_dir, key=classic_key)
assert num_sub.dir.is_dir()

# Create random dataframe
n_rows = 100
targets = "prediction_mymodel"
test_dataf = pd.DataFrame(np.random.uniform(size=n_rows), columns=[targets])
test_dataf["id"] = [uuid.uuid4() for _ in range(n_rows)]
test_dataf = test_dataf.set_index("id")
test_dataf.head(2)

No existing directory found at 'test_sub'. Creating directory...

,prediction_mymodel
id,
3c5a588a-a4e1-4b4d-976a-e86079ea5053,0.704015
d9ea0000-47a6-4e68-985f-36b3f9c51be4,0.667652


CSVs can be saved with `.save_csv`. [`NumeraiClassicSubmitter`](https://crowdcent.github.io/numerblox/submission.html#numeraiclassicsubmitter) will automatically provide checks to make sure that data is saved correctly.

In [ ]:
file_name = "test.csv"
num_sub.save_csv(dataf=test_dataf, file_name=file_name, cols=targets)
num_sub.save_csv(dataf=test_dataf, file_name="test2.csv", cols=targets)
pd.read_csv(f"{test_dir}/{file_name}").head(2)

📄 Saving predictions CSV to 'test_sub/test.csv'. 📄

📄 Saving predictions CSV to 'test_sub/test2.csv'. 📄

,id,prediction
0,3c5a588a-a4e1-4b4d-976a-e86079ea5053,0.704015
1,d9ea0000-47a6-4e68-985f-36b3f9c51be4,0.667652


[`NumeraiClassicSubmitter`](https://crowdcent.github.io/numerblox/submission.html#numeraiclassicsubmitter) also gives you the option to combine multiple predictions csvs that you already created. Prediction will be standardized by default.

In [3]:
#| output: asis
#| echo: false
show_doc(NumeraiClassicSubmitter.combine_csvs)

---

[source](https://github.com/crowdcent/numerblox/blob/master/numerblox/submission.py#L100){target="_blank" style="float:right; font-size:smaller"}

### BaseSubmitter.combine_csvs

>      BaseSubmitter.combine_csvs (csv_paths:list, aux_cols:list,
>                                  era_col:str=None, pred_col:str='prediction')

Read in csv files and combine all predictions with a rank mean. 

Multi-target predictions will be averaged out. 

:param csv_paths: List of full paths to .csv prediction files. 

:param aux_cols: ['id'] for Numerai Classic. 

['ticker', 'last_friday', 'data_type'], for example, with Numerai Signals. 

:param era_col: Column indicating era ('era' or 'last_friday'). 

Will be used for Grouping the rank mean if given. Skip groupby if no era_col provided. 

:param pred_col: 'prediction' for Numerai Classic and 'signal' for Numerai Signals.

In [ ]:
combined = num_sub.combine_csvs(["test_sub/test.csv", "test_sub/test2.csv"], aux_cols=['id'])
assert combined.columns == ['prediction']
combined.head(2)

  0%|          | 0/2 [00:00<?, ?it/s]

,prediction
id,
3c5a588a-a4e1-4b4d-976a-e86079ea5053,0.75
d9ea0000-47a6-4e68-985f-36b3f9c51be4,0.71


In [ ]:
#| include: false
def test_signal_validity(
        submitter: NumeraiClassicSubmitter, dataf: pd.DataFrame
):
    """ Test value range of prediction. """
    try:
        invalid_signal = deepcopy(dataf)
        invalid_signal.iloc[0]["prediction_mymodel"] += 10
        submitter.save_csv(
            invalid_signal,
            file_name="should_not_save.csv",
            cols="prediction_mymodel",
        )
    except ValueError:
        return True
    return False

assert test_signal_validity(num_sub, test_dataf)

Uncomment to save CSV and upload predictions in one go.

In [ ]:
# Full submission
# num_sub.full_submission(dataf=test_dataf, file_name='test.csv', cols=targets, model_name="test")

After a successful submission, contents can be removed to keep a clean environment.

In [ ]:
num_sub.remove_base_directory()
assert not os.path.exists(test_dir)

⚠ Deleting directory for 'NumeraiClassicSubmitter' ⚠
Path: '/home/clepelaars/numerblox/nbs/test_sub'

## 2. NumeraiSignalsSubmitter

Numerai Signals submissions. Uses [SignalsAPI](https://numerapi.readthedocs.io/en/latest/_modules/numerapi/signalsapi.html) under the hood.

In [4]:
#| echo: false
#| output: asis
show_doc(NumeraiSignalsSubmitter)

---

[source](https://github.com/crowdcent/numerblox/blob/master/numerblox/submission.py#L215){target="_blank" style="float:right; font-size:smaller"}

### NumeraiSignalsSubmitter

>      NumeraiSignalsSubmitter (directory_path:str, key:numerblox.key.Key,
>                               *args, **kwargs)

Submit for Numerai Signals.

:param directory_path: Base directory to save and read prediction files from. 

:param key: Key object containing valid credentials for Numerai Signals. 

*args, **kwargs will be passed to SignalsAPI initialization.

### Example usage 2: NumeraiSignalsSubmitter

Initialization (Random credentials)

In [ ]:
test_dir_signals = "test_sub_signals"
signals_key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
signals_sub = NumeraiSignalsSubmitter(directory_path=test_dir_signals, key=signals_key)
assert signals_sub.dir.is_dir()

No existing directory found at 'test_sub_signals'. Creating directory...

In [ ]:
def create_random_signals_dataf(n_rows=5000):
    signals_test_dataf = pd.DataFrame(
        np.random.uniform(size=(n_rows, 1)), columns=["signal"]
    )
    signals_test_dataf["ticker"] = [
        "".join(choices(ascii_uppercase, k=4)) for _ in range(n_rows)
    ]
    last_friday = str((datetime.now() + relativedelta(weekday=FR(-1))).date()).replace("-", "")
    signals_test_dataf['last_friday'] = last_friday
    signals_test_dataf['data_type'] = 'live'
    return signals_test_dataf

signals_test_dataf = create_random_signals_dataf()
signals_test_dataf.head(2)

,signal,ticker,last_friday,data_type
0,0.907471,XONJ,20230901,live
1,0.149298,KDHF,20230901,live


In [ ]:
#| include: false
signals_cols = ["signal", "ticker", "data_type", "last_friday"]
file_name = "signals_test.csv"
signals_sub.save_csv(dataf=signals_test_dataf, file_name=file_name, cols=signals_cols)
signals_sub.save_csv(dataf=signals_test_dataf, file_name="signals_test2.csv", cols=signals_cols)
pd.read_csv(f"{test_dir_signals}/{file_name}").head(2)

📄 Saving Signals predictions CSV to 'test_sub_signals/signals_test.csv'. 📄

📄 Saving Signals predictions CSV to 'test_sub_signals/signals_test2.csv'. 📄

,signal,ticker,data_type,last_friday
0,0.907471,XONJ,live,20230901
1,0.149298,KDHF,live,20230901


In [ ]:
#| include: false
combined_signals = signals_sub.combine_csvs(csv_paths=["test_sub_signals/signals_test.csv",
                                               "test_sub_signals/signals_test2.csv"],
                                    aux_cols=['ticker', 'last_friday', 'data_type'],
                                    era_col='last_friday',
                                    pred_col='signal')
assert combined_signals.columns == ['signal']
combined_signals.head(2)

  0%|          | 0/2 [00:00<?, ?it/s]

,,,signal
ticker,last_friday,data_type,
XONJ,20230901,live,0.9088
KDHF,20230901,live,0.1594


Saving Signals CSV should fail if there is no valid ticker column or if `signal` has values outside the range $[0...1]$.

In [ ]:
#| include: false
def test_signal_validity(
    submitter: NumeraiSignalsSubmitter, signals_dataf: pd.DataFrame
):
    """ Test value range of signal. """
    try:
        invalid_signal = deepcopy(signals_dataf)
        invalid_signal.loc[0, "signal"] += 10
        submitter.save_csv(
            invalid_signal,
            file_name="should_not_save.csv",
            cols=list(invalid_signal.columns),
        )
    except ValueError:
        return True
    return False


def test_ticker_validity(
    submitter: NumeraiSignalsSubmitter, signals_dataf: pd.DataFrame
):
    """ Test safeguard if ticker column is not valid. """
    try:
        invalid_ticker = deepcopy(signals_dataf)
        invalid_ticker = invalid_ticker.rename(
            {"ticker": "not_a_valid_ticker_format"}, axis=1
        )
        submitter.save_csv(
            invalid_ticker,
            file_name="should_not_save.csv",
            cols=list(invalid_ticker.columns),
        )
    except NotImplementedError:
        return True
    return False

assert test_signal_validity(signals_sub, signals_test_dataf)
assert test_ticker_validity(signals_sub, signals_test_dataf)

Uncomment to save CSV and upload predictions in one go.

In [ ]:
# Full Signals submission
# signals_sub.full_submission(dataf=signals_test_dataf, file_name='signals_test.csv', cols=signals_cols, model_name="test")

After a successful submission, contents can be removed to keep a clean environment.

In [ ]:
signals_sub.remove_base_directory()
assert not os.path.exists(test_dir_signals)

⚠ Deleting directory for 'NumeraiSignalsSubmitter' ⚠
Path: '/home/clepelaars/numerblox/nbs/test_sub_signals'

## 3. NumerBaySubmitter

Wrapper on top of the tournament submitters, submits to [NumerBay](https://numerbay.ai/) to fulfill sale orders. Make sure you have numerbay installed (`pip install numerbay`).

In [5]:
#| echo: false
#| output: asis
show_doc(NumerBaySubmitter)

---

[source](https://github.com/crowdcent/numerblox/blob/master/numerblox/submission.py#L280){target="_blank" style="float:right; font-size:smaller"}

### NumerBaySubmitter

>      NumerBaySubmitter (tournament_submitter:Union[__main__.NumeraiClassicSubm
>                         itter,__main__.NumeraiSignalsSubmitter],
>                         upload_to_numerai:bool=True,
>                         numerbay_username:str=None,
>                         numerbay_password:str=None)

Submit to NumerBay to fulfill sale orders, in addition to submission to Numerai.

:param tournament_submitter: Base tournament submitter (NumeraiClassicSubmitter or NumeraiSignalsSubmitter). This submitter will use the same directory path.
:param upload_to_numerai: Whether to also submit to Numerai using the tournament submitter. Defaults to True, set to False to only upload to NumerBay.
:param numerbay_username: NumerBay username
:param numerbay_password: NumerBay password

### Example usage 3: NumerBaySubmitter

Numerai submissions (existing file)

In [ ]:
# numerai_submitter = NumeraiClassicSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=numerai_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.upload_predictions(file_name='upload.csv', model_name='mymodel',
#                                 numerbay_product_full_name='numerai-predictions-myproduct')

Numerai submissions (from NumerFrame)

In [ ]:
# numerai_submitter = NumeraiClassicSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=numerai_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.full_submission(dataf, file_name='upload.csv', model_name='mymodel',
#                              numerbay_product_full_name='numerai-predictions-myproduct',
#                              cols='some_pred_col')

Signals submissions (existing file)

In [ ]:
# signals_submitter = NumeraiSignalsSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=signals_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.upload_predictions(file_name='upload-signals.csv', model_name='mymodel',
#                                 numerbay_product_full_name='signals-predictions-myproduct')

Signals submissions (from NumerFrame)

In [ ]:
# signals_submitter = NumeraiSignalsSubmitter(directory_path="/app/notebooks/tmp", key=key)
# nb_submitter = NumerBaySubmitter(tournament_submitter=signals_submitter, upload_to_numerai=False,
#                                  numerbay_username="someusername", numerbay_password="somepassword")
# nb_submitter.full_submission(dataf, file_name='upload-signals.csv', model_name='mymodel',
#                              numerbay_product_full_name='signals-predictions-myproduct',
#                              cols=['bloomberg_ticker', 'friday_date', 'data_type', 'signal'])

------------------------------------------------------------